In [4]:
def figure3(P):
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(12,7))
    ax = plt.subplot(1, 2, 1, aspect='equal')
    P.tri.plot(edgecolor='black', lw = 0.3)
    P.tri.plot_centroids(ax=ax, marker='o', markersize = '1.', color='red')   
    for i in trinodes: ax.plot(i[0], i[1], 'o', ms = 2, color = 'black')
    for i in range(P.npump): ax.plot(P.xypumpbores[i], ms = 5, color = 'blue', zorder=2.)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title('Triangular Mesh (Nodes = %i)' %len(P.cell2dtri))
    
    ax = plt.subplot(1, 2, 2, aspect='equal')
    P.vor.plot(ax=ax, edgecolor='black', lw = 0.4)
    for i in P.xcycvor: ax.plot(i[0], i[1], 'o', color = 'green', ms = 1.5)
    for i in vornodes: ax.plot(i[0], i[1], 'o', ms = 2, color = 'black')
    for i in range(P.npump): ax.plot(P.xypumpbores[i], ms = 2, color = 'blue', zorder=2.)
    ax.set_xlabel('X (m)')
    ax.set_yticks([])
    ax.set_yticklabels([])
    ax.set_title('Voronoi Mesh (Nodes = %i)' %len(P.cell2dvor))
    
    plt.subplots_adjust(wspace=0.05, hspace=0.1)    
    plt.tight_layout()  
    plt.show()   
    fig.savefig('../figures/fig3.tif', dpi=300)
    


In [5]:
# PINCHOUT SCENARIO
def figure5(P, models, X0, Y0, X1, Y1, vmin = None, vmax = None): # array needs to be a string of a property eg. 'k11', 'angle2'
    import matplotlib.pyplot as plt
    P1_L = 500 #(500**2 + 500**2)**0.5
    P2_L = 2500 #(2500**2 + 2500**2)**0.5
    CP1_L = 3000 #(3000**2 + 3000**2)**0.5

    c = mpl.colors.ListedColormap(['wheat', 'lightgreen', 'coral'])
    # wheat #fbec99 saturdation 100
    # coral #fc9a58 saturdation 150
    # lightgreen #91ff95
    n = mpl.colors.Normalize(vmin=-1,vmax=1)
    
    fig = plt.figure(figsize = (10,10))
    #fig.suptitle("TRANSECT - " + array)
    nm = [0,2,4]
    for i in nm:
        ax = plt.subplot(5, 2, i+1)
        M = models[i]
        model = M.gwf
        a = getattr(M, 'k11')
        xsect = flopy.plot.PlotCrossSection(model=model, line={"line": [(X0, Y0),(X1, Y1)]}, 
                                            #extent = [P.x0,P.x1,P.z0,P.z1], 
                                            geographic_coords=True)
        csa = xsect.plot_array(a = logfunc(a), cmap=c, norm=n, alpha=0.8, vmin = vmin, vmax = vmax)
        ax.set_ylim([-500, 0])
        
        linecollection = xsect.plot_grid(lw = 0.2, color = 'black') # Don't plot grid for reference
        #plt.colorbar(csa, shrink = 0.7)
        
        cb_ax = fig.add_axes([.44,.44,.007,0.1])
        fig.colorbar(csa,orientation='vertical',cax=cb_ax, label='Log Kh')
        
        
        if i == 0:
            ax.set_title('a) Log Kh', y=1.1, size = 11)
            ax.plot(P1_L, -50, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(P1_L, -350, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(P1_L, -450, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(P2_L, -50, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(P2_L, -150, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(P2_L, -250, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(CP1_L, -250, color='black', marker='o', markerfacecolor='red', markersize=8)
            ax.annotate("P1",(P1_L, 20), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("P2",(P2_L, 20), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("CP1",(CP1_L, 20), size = 10, ha = 'center', annotation_clip = False)
        if i == 0:
            ax.set(xticklabels=[])
        if i == 2:
            ax.set_ylabel('z (m)', size = 10)
            ax.set(xticklabels=[])
        if i == 4: 
            ax.set_xlabel('x (km)', size = 10)
            #ax.set_xticks([0, 2000, 4000, 6000])
            #ax.set_xticklabels([0, 2, 4, 6])
        
    for i in nm:
        ax = plt.subplot(5, 2, i+2)
        M = models[i]
        model = M.gwf
        a = getattr(M, 'head_ss')
        xsect = flopy.plot.PlotCrossSection(model=model, line={"line": [(X0, Y0),(X1, Y1)]}, 
                                            extent = [P.x0,P.x1,P.z0,P.z1], geographic_coords=True)
        csa = xsect.plot_array(a = a, cmap='Spectral', alpha=0.8, vmin = -45, vmax = 10)
        ax.set_ylim([-500, 0])
        ax.annotate(str('CPz = ' + M.modelname[3::]), (5800, -400), size = 10, 
                    horizontalalignment='right', bbox=dict(facecolor = 'white'))
        if i == 0:
            ax.set(xticklabels=[])
            ax.set(yticklabels=[])
            ax.set_title('b) Head Distribution (Transect)', y = 1.1, size = 11)
        if i == 2:
            ax.set(xticklabels=[])
            ax.set(yticklabels=[])
        if i == 4: 
            ax.set_xlabel('x (km)', size = 10)
            ax.set_xticks([0, 2000, 4000, 6000])
            ax.set_xticklabels([0, 2, 4, 6])
            ax.set(yticklabels=[])
        #plt.colorbar(csa, shrink = 0.7)
        
    for i in nm:
        layer = 0
        ax = plt.subplot(3, 3, int(i/2 + 7))
        M = models[i]
        model = M.gwf
        water_table = flopy.utils.postprocessing.get_water_table(M.gwf.output.head().get_data())
        M.heads_disv = -1e30 * np.ones_like(M.idomain, dtype=float) 
        for k, h in enumerate(water_table):
            if math.isnan(h) == False: 
                M.heads_disv[M.cellid_disu==k] = h        
        pmv = flopy.plot.PlotMapView(modelgrid=M.vgrid)
        H = pmv.plot_array(M.heads_disv[layer], vmin = -45, vmax = 10, cmap = 'Spectral', alpha = 0.6)
        
        if M.plan == 'car': P.sg.plot(ax=ax, edgecolor='black', lw = 0.2)
        if M.plan == 'tri': P.tri.plot(ax=ax, edgecolor='black', lw = 0.2)
        if M.plan == 'vor': P.vor.plot(ax=ax, edgecolor='black', lw = 0.2)
        if i == 2: ax.set_yticklabels([])
        if i == 2: ax.set_xlabel('x (km)', size = 10)
        if i == 4: ax.set(yticklabels=[])
        if i == 0:
            #ax.set_yticks([0, 2000, 4000, 6000])
            #ax.set_yticklabels([0, 2, 4, 6])
            ax.plot(500,500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(2500,2500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(3000,3000, color='black', marker='o', markerfacecolor='red', markersize=8)
            ax.annotate("P1",(500,500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("P2",(2500,2500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("CP1",(3000,3000+200), size = 10, ha = 'center', annotation_clip = False)
            ax.plot([X0,X1],[Y0,Y1],'--', color = 'black', lw = 0.7)
            #for j in range(len(P.xyobsbores)):
            #    ax.plot(P.xyobsbores[j][0], P.xyobsbores[j][1],'o', ms = '4', c = 'black')
            #    ax.annotate(P.idobsbores[j], (P.xyobsbores[j][0], P.xyobsbores[j][1]+100), c='black', size = 12) #, weight = 'bold')
        
            #for j in range(len(P.xypumpbores)):
            #    ax.plot(P.xypumpbores[j][0], P.xypumpbores[j][1],'o', ms = '4', c = 'red')
            #    ax.annotate(P.idpumpbores[j], (P.xypumpbores[j][0], P.xypumpbores[j][1]+100), c='red', size = 12) #, weight = 'bold')
        
        if i == 2: 
            ax.set_title('c) Head Distribution (Layer = 0)', y = 1.01, size = 11)
        else: 
            ax.set_title(' ') 
        ax.set_xticks([0, 2000, 4000, 6000])
        ax.set_xticklabels([0, 2, 4, 6])
        #plt.colorbar(H, shrink = 0.7)
        ax.annotate(str('CPz = ' + M.modelname[3::]), (3000, 400), size = 10, 
                    horizontalalignment='center', bbox=dict(facecolor = 'white'))
    cb_ax = fig.add_axes([.91,.124,.007,0.2])
    fig.colorbar(H,orientation='vertical',cax=cb_ax, label = 'Head (m)')


    plt.subplots_adjust(wspace=0.05, hspace=0.1)    
    plt.tight_layout()  
    plt.show()   
    fig.savefig('../figures/fig5.tif', dpi=300)


In [6]:
# FAULT SCENARIO
def figure6(P, models, X0, Y0, X1, Y1, vmin = None, vmax = None): # array needs to be a string of a property eg. 'k11', 'angle2'
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import flopy
    import math
    logfunc = lambda e: np.log10(e)
    
    c = mpl.colors.ListedColormap(['wheat', 'lightgreen', 'coral'])
    n = mpl.colors.Normalize(vmin=-1,vmax=1)
    
    fig = plt.figure(figsize = (10,10))
    #fig.suptitle("TRANSECT - " + array)
    nm = [0,2,4]
    for i in nm:
        ax = plt.subplot(5, 2, i+1)
        M = models[i]
        model = M.gwf
        a = getattr(M, 'k11')
        xsect = flopy.plot.PlotCrossSection(model=model, line={"line": [(X0, Y0),(X1, Y1)]}, 
                                            extent = [P.x0,P.x1,P.z0,P.z1], geographic_coords=True)
        
            
        csa = xsect.plot_array(a = logfunc(a), cmap=c, norm=n, alpha=0.8) #cmap=c, norm=n, 
        ax.set_ylim([-500, 0])
        
        linecollection = xsect.plot_grid(lw = 0.2, color = 'black') # Don't plot grid for reference
        #plt.colorbar(csa, shrink = 0.7)
        cb_ax = fig.add_axes([.44,.44,.007,0.1])
        fig.colorbar(csa,orientation='vertical',cax=cb_ax, label='Log Kh')
        
        if i == 0:
            ax.set_title('a) Log Kh', y=1.01, size = 11)
            #ax.plot(P1_L, -50, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            #ax.annotate("P1",(P1_L, 20), size = 10, ha = 'center', annotation_clip = False)
        if i == 0:
            ax.set(xticklabels=[])
        if i == 2:
            ax.set_ylabel('z (m)', size = 10)
            ax.set(xticklabels=[])
        if i == 4: 
            ax.set_xlabel('x (km)', size = 10)
            ax.set_xticks([0, 2000, 4000, 6000])
            ax.set_xticklabels([0, 2, 4, 6])
        
    for i in nm:
        ax = plt.subplot(5, 2, i+2)
        M = models[i]
        model = M.gwf
        a = getattr(M, 'head_ss')
        xsect = flopy.plot.PlotCrossSection(model=model, line={"line": [(X0, Y0),(X1, Y1)]}, 
                                            extent = [P.x0,P.x1,P.z0,P.z1], geographic_coords=True)
        csa = xsect.plot_array(a = a, cmap='Spectral', alpha=0.8, vmin = vmin, vmax = vmax)
        ax.set_ylim([-500, 0])
        ax.annotate(str('FD = ' + M.modelname[3::]), (5800, -400), size = 10, 
                    horizontalalignment='right', bbox=dict(facecolor = 'white'))
        if i == 0:
            ax.set(xticklabels=[])
            ax.set(yticklabels=[])
            ax.set_title('b) Head Distribution (Transect)', y = 1.01, size = 11)
        if i == 2:
            ax.set(xticklabels=[])
            ax.set(yticklabels=[])
        if i == 4: 
            ax.set_xlabel('x (km)', size = 10)
            ax.set_xticks([0, 2000, 4000, 6000])
            ax.set_xticklabels([0, 2, 4, 6])
            ax.set(yticklabels=[])
        #plt.colorbar(csa, shrink = 0.7)
        
    for i in nm:
        layer = 0
        ax = plt.subplot(3, 3, int(i/2 + 7))
        M = models[i]
        model = M.gwf
        water_table = flopy.utils.postprocessing.get_water_table(M.gwf.output.head().get_data())
        M.heads_disv = -1e30 * np.ones_like(M.idomain, dtype=float) 
        for k, h in enumerate(water_table):
            if math.isnan(h) == False: 
                M.heads_disv[M.cellid_disu==k] = h        
        pmv = flopy.plot.PlotMapView(modelgrid=M.vgrid)
        H = pmv.plot_array(M.heads_disv[layer], cmap = 'Spectral', alpha = 0.6, vmin = vmin, vmax = vmax) #
        
        if M.plan == 'car': P.sg.plot(ax=ax, edgecolor='black', lw = 0.2)
        if M.plan == 'tri': P.tri.plot(ax=ax, edgecolor='black', lw = 0.2)
        if M.plan == 'vor': P.vor.plot(ax=ax, edgecolor='black', lw = 0.2)
        if i == 2: ax.set_yticklabels([])
        if i == 2: ax.set_xlabel('x (km)', size = 10)
        if i == 4: ax.set(yticklabels=[])
        if i == 0:
            ax.set_yticks([0, 2000, 4000, 6000])
            ax.set_yticklabels([0, 2, 4, 6])
            ax.plot(500,500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(500,5500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(5500,500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.plot(5500,5500, color='black', marker='o', markerfacecolor='lightblue', markersize=8)
            ax.annotate("P1",(500,500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("P2",(500,5500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("P3",(5500,500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.annotate("P4",(5500,5500+200), size = 10, ha = 'center', annotation_clip = False)
            ax.plot([X0,X1],[Y0,Y1],'--', color = 'black', lw = 0.7)
            #for j in range(len(P.xyobsbores)):
            #    ax.plot(P.xyobsbores[j][0], P.xyobsbores[j][1],'o', ms = '4', c = 'black')
            #    ax.annotate(P.idobsbores[j], (P.xyobsbores[j][0], P.xyobsbores[j][1]+100), c='black', size = 12) #, weight = 'bold')
        
            #for j in range(len(P.xypumpbores)):
            #    ax.plot(P.xypumpbores[j][0], P.xypumpbores[j][1],'o', ms = '4', c = 'red')
            #    ax.annotate(P.idpumpbores[j], (P.xypumpbores[j][0], P.xypumpbores[j][1]+100), c='red', size = 12) #, weight = 'bold')
        if i == 2: 
            ax.set_title('c) Head Distribution (Layer = 0)', y = 1.01, size = 11)
        else:
            ax.set_title(' ') 
        ax.set_xticks([0, 2000, 4000, 6000])
        ax.set_xticklabels([0, 2, 4, 6])
        #plt.colorbar(H, shrink = 0.7)
        ax.annotate(str('FD = ' + M.modelname[3::]), (3000, 400), size = 10, 
                    horizontalalignment='center', bbox=dict(facecolor = 'white'))
    cb_ax = fig.add_axes([.91,.124,.007,0.2])
    fig.colorbar(H,orientation='vertical',cax=cb_ax, label = 'Head (m)')


    plt.subplots_adjust(wspace=0.05, hspace=0.1)    
    plt.tight_layout()  
    plt.show()   
    fig.savefig('../figures/fig6.tif', dpi=300)

In [ ]:
def figure7(P, std_PO, std_FD):
    
    fig = plt.figure(figsize=(12,5))
    fig.suptitle('Standard deviation in head due to alternative structural models')
    ax = plt.subplot(1, 2, 1, aspect='equal')
    #P.vor.plot(ax=ax, edgecolor='black', lw = 0.2)
    pmv = flopy.plot.PlotMapView(model=M.gwf)
    pmv.plot_grid(edgecolor='black', lw = 0.2)
    A = pmv.plot_array(var_PO, cmap = 'Spectral', alpha = 0.6)#vmin = vmin, vmax = vmax, 
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title('Pinchout Scenario')
    plt.colorbar(A, ax=ax, shrink = 0.4)
    
    ax = plt.subplot(1, 2, 2, aspect='equal')
    #P.vor.plot(ax=ax, edgecolor='black', lw = 0.2)
    grid = N.gwf.modelgrid
    pmv = flopy.plot.PlotMapView(model=N.gwf)
    pmv.plot_grid(edgecolor='black', lw = 0.2)
    B = pmv.plot_array(var_FD, cmap = 'Spectral', alpha = 0.6)#vmin = vmin, vmax = vmax, 
    
    ax.set_xlabel('X (m)')
    ax.set_yticks([])
    ax.set_yticklabels([])
    ax.set_title('Fault Scenario')
    plt.colorbar(B, ax=ax, shrink = 0.4)
    
    plt.subplots_adjust(wspace=0.05, hspace=0.1)    
    plt.tight_layout()  
    plt.show()   
    fig.savefig('../figures/fig7.tif', dpi=300)